# Recipe Shopping List

This notebook generates a shopping list for recipe ingredient list specified in a spreadsheet saved as TSV (tab).

In [94]:
open System.IO
open System.Text.RegularExpressions

type RecipeIngredient =
    {
        Recipe : string
        Ingredient : string
        Quantity : float
        Unit : string
    }

//matches a numeric decimal quantity followed by a non numeric unit identifier, e.g. "1.5oz"
let quantityRegex = new Regex( @"([0-9\.]+)\s*(\D+)")

//Map of recipe name to list of ingredients
let db = 
    Directory.GetFiles(".","*.tsv").[0] //will take the first file we find
    |> File.ReadAllLines
    |> Array.skip 1                     //skip header
    |> Array.map( fun row ->
        let s = row.Split('\t')
        
        //last col mixes quantity and unit
        let m = quantityRegex.Match(s.[2])
        
        //if we have quantity and unit (e.g. 1.5oz)
        if m.Groups.Count = 3 then
            { Recipe=s.[0]; Ingredient=s.[1]; Quantity=m.Groups.[1].Value|>float; Unit=m.Groups.[2].Value }
        //unit is identity (e.g. 1 carrot)
        else
            { Recipe=s.[0]; Ingredient=s.[1]; Quantity=s.[2]|>float; Unit=""}
    )
    |> Array.groupBy( fun ri -> ri.Recipe )
    |> Map.ofArray
    
//display the recipes we know about
let format ( s : string ) = s.Replace(";",";\n")
db |> Map.toArray |> Array.map fst |> sprintf "%A" |> format

"[|"baked ziti chickpeas";
 "cabbage cheddar pie";
 "celery curry";
 "chana masala";

  "cream of lentil soup";
 "cuban black beans";
 "edamame fried rice";

  "edamame noodles";
 "lentils with fruit and sweet potatoes";
 "okra curry";

  "penne all'arrabbiata";
 "portabello mushroom ragout";
 "potato chilli burro";

  "spinach enchiladas verde";
 "spinach lasagne";
 "spinach lentil soup";

  "sweet potato beetroot curry";
 "sweet potato black bean quessadillas";

  "sweet potato burritos";
 "szechuan tofu broccoli mushrooms";

  "two step southwestern stew";
 "veggie burros";
 "zucchini tostadas"|]"

**Copy the list above below and remove the recipes you don't want**
(TODO checkbox GUI)

In [97]:
let recipes =
    [|"baked ziti chickpeas";
 "celery curry";
 "chana masala";

  "cream of lentil soup";
 "cuban black beans";
 "edamame fried rice";

 "lentils with fruit and sweet potatoes";
 "okra curry";

  "penne all'arrabbiata";
 "portabello mushroom ragout";
 "potato chilli burro";

  "spinach enchiladas verde";
 "spinach lasagne";
 "spinach lentil soup";

  "sweet potato beetroot curry";
 "sweet potato black bean quessadillas";

 "szechuan tofu broccoli mushrooms";

  "two step southwestern stew";
 "veggie burros";
 "zucchini tostadas"|]
 
//-----------------------------
// given selected recipes, collect all ingredients as groups, and sum their quantities
let ingredients =
    recipes
    
    //index by ingredient
    |> Array.collect( fun r ->
        db.[r] |> Array.map( fun ri -> ri.Ingredient,ri )
    )
    |> Array.groupBy fst
    
    //aggregate quantities for each ingredient as much as possilbe
    |> Array.map( fun (i,tuples) -> 
    
        //sum quantities of the same unit
        let unitQuantities = 
            tuples
            |> Array.map snd
            |> Array.groupBy( fun ri2 -> ri2.Unit ) 
            |> Array.map( fun (u,ris2) ->
                (ris2 |> Array.sumBy( fun ri2 -> ri2.Quantity ) |> string) + " " + u
            )
        //map to string
        i + ":" + (unitQuantities |> String.concat "; ")
    )
    
ingredients |> String.concat "\n"

"penne pasta:24 oz
zucchini:8 
chickpeas:4 can
tomatoes basil garlic oregano:1 can
tomato puree:1 c
kalamata olives:0.25 c
olive oil:13 tb
tomato paste:1 tb
garlic:25 clove
breadcrumbs:2 tb
cauliflower:1 
celery:1 
ginger:4 tb; 6 in
tomatoes green chillies:2 c; 3 can
rice:5.5 c
onion:8 
vegetable oil:22 tb
cumin:1 tb; 4.5 t
cilantro:0.75 c
chillis:2 
corriander:1 tb; 1 t
chilli powder:5 t
tumeric:1.5 t
curry powder:5 t
tomatoes:3 can
garam masala:1 t
lemon:2 
lentils:2.5 c; 16 oz
chipotle adobo sauce:1 t
butter:1 tb
cream of mushroom soup:36 oz
banana:3 
green bell pepper:2 
stock cubes:3 
black beans:5 can
green onion:7 
carrot:6 
yellow bell pepper:1 
green onions:12 
soy sauce:4 tb; 0.25 c
edamame:18 oz
pineapple:1 can
pear:2 
sweet potato:1 large; 5 
plantain:1 
okra:6 c
cherry tomatoes:8 oz
cumin seeds:2 t
paprika:0.5 t
crushed tomatoes:1 can
diced tomatoes:1 can
red pepper flakes:1 tb
lima beans:10 oz
green beans:1 can
portabello:1.5 c
stewed tomatoes:1 can
feta:1 c
potatos:3 
gr